In [ ]:
import pandas as pd


import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import time
import h5py
import sklearn
import copy
from sklearn.utils import shuffle
from scipy import stats

import os
from plottingFunctions import *
from modelFunctions import *

print('Tensorflow Version {}'.format(tf.__version__))

In [ ]:
start_notebook=time.time()

In [ ]:
if not os.path.isdir("models"):
    os.mkdir("models")
    
if not os.path.isdir("plots"):
    os.mkdir("plots")

## Hyperparameters

In [ ]:
class HParams(object):
    def __init__(self):
        self.Lambda = 10
        self.batchsize = 50
        self.trainingBatchSize = 50
        self.thetaBins = 6
        self.trainingEpochs = 20
        self.adversary_lr = 1e-3
        self.classifier_lr = 1e-3
        self.nUpdates_Classifier = 10
        self.valSplit = 0.9

HPARAMS = HParams()

### More Parameters

```
features = ['pt', 'eta', 'phi', 'mass', 'theta', 'radii', 'z', 'tau1', 'tau2', 'labels']
```

In [ ]:
classifierOnly = False
alpha = 0.5

features = ['theta', 'radii', 'z', 'tau1', 'tau2']

## Models

In [ ]:
def build_classifier(inputs, concatenated_layers):
    
        
    x = tf.keras.layers.Dense(128, activation='relu')(concatenated_layers)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    
    
    output = tf.keras.layers.Dense(2, activation='softmax', name = 'labels')(x) 
    model = tf.keras.Model(inputs=inputs, outputs=output, name='Classifier')
    
    
    return model

In [ ]:
def build_adversary(inputs, concatenated_layers):

    x = tf.keras.layers.Dense(64, activation='relu', name='Adversary_Dense64')(concatenated_layers)
    x = tf.keras.layers.Dense(128, activation='relu', name='Adversary_Dense128')(x)

    inputs = [input_from_model, feature_input]
    output = tf.keras.layers.Dense(HPARAMS.thetaBins, activation='softmax', name = 'output')(x) 
        
    model = tf.keras.Model(inputs=inputs, outputs=output, name='Adversary')

    return model

## Get datasets as dictionaries
### Datasets generated with ToyModel/makeFourVectors.ipynb

In [ ]:
trainfilename = 'data/jetConstTrain_overlap.npz'
testfilename = 'data/jetConstTest_overlap.npz'

# created by running makeFourVectors.ipynb with addPerturbation = False
data_train = np.load(trainfilename)
data_test = np.load(testfilename)

# created by running makeFourVectors.ipynb with addPerturbation = True
data_train_shift = np.load(trainfilename.replace('.npz', '_perturb.npz'))
data_test_shift = np.load(testfilename.replace('.npz', '_perturb.npz'))
    
feat_list = features
feat_xaug = []

Nlist = len(feat_list)
Nxaug = len(feat_xaug)
nEvents = len(data_train['pt'][:,0].flatten())

print()
print('Number of particle list features: ', Nlist)
print('Number of XAUG features: ', Nxaug)
print('N Events: ', nEvents)


### Get dictionaries

In [ ]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = getXY(data_train,
                                                       data_test,
                                                       features,
                                                       HPARAMS.valSplit,
                                                       nEvents,
                                                      )


X_train_shift, Y_train_shift, X_test_shift, Y_test_shift, X_val_shift, Y_val_shift = getXY(data_train_shift,
                                                                                           data_test_shift,
                                                                                           features,
                                                                                           HPARAMS.valSplit,
                                                                                           nEvents,
                                                                                          )

In [ ]:
layer_names = [key for key in X_train.keys() if not 'labels' in key]
layer_names

## Make $\mathrm{log}(p_{T})$ and $\mathrm{log}(z)$

In [ ]:
try:
    X_train['pt'] = np.log(X_train['pt'])
    X_test['pt'] = np.log(X_test['pt'])
    X_val['pt'] = np.log(X_val['pt'])
except:
    pass

try:
    X_train['z'] = np.log(X_train['z'])
    X_test['z'] = np.log(X_test['z'])
    X_val['z'] = np.log(X_val['z'])
except:
    pass



In [ ]:
try:
    plotFeature(X_train, Y_train, 'theta', alpha)
except:
    pass

In [ ]:
try:
    plotFeature(X_train, Y_train, 'z', alpha)
except:
    pass

In [ ]:
try:
    plotFeature(X_train, Y_train, 'radii', alpha)
except:
    pass

In [ ]:
try:
    plotFeature(X_train, Y_train, 'tau1', alpha)
except:
    pass

In [ ]:
try:
    plotFeature(X_train, Y_train, 'tau2', alpha)
except:
    pass

## Signal difference adversary
Note: This is not currently used as an input

In [ ]:
plotFeature(X_train, Y_train, 'theta', alpha)

In [ ]:
plotFeature(X_train_shift, Y_train_shift, 'theta', alpha)

In [ ]:
X_adv, Y_adv = getAdversary(X_train['theta'], X_train_shift['theta'], Y_train)

## Bin $\Delta \theta$
`binned_theta` is an input to the adversary model and
`theta_labels` are the labels for the adversary model

In [ ]:
thetabins = np.linspace(0, 0.3, HPARAMS.thetaBins)
binned_theta = np.digitize(X_train['theta'], thetabins) - 1
theta_labels = tf.one_hot(binned_theta.squeeze(), HPARAMS.thetaBins)

In [ ]:
print('\nfirst 5 bins\n')
print(binned_theta[:5])
print('\nfirst 5 labels\n')
print(theta_labels.numpy()[:5])

In [ ]:
plt.hist(X_train['theta'].flatten(), bins=thetabins, histtype='step', density=True)
plt.xlabel(r'$\Delta \theta$')
plt.ylabel('Density')
plt.title(r'Binned $\Delta \theta$')
plt.show()

## Classifier and Adversary Training Loop



### Get datasets

In [ ]:
train_dataset = getDataset(X_train, Y_train, HPARAMS.batchsize)
valid_dataset = getDataset(X_val, Y_val, HPARAMS.batchsize)
train_dataset_adv = getDataset(X_train['theta'], theta_labels, HPARAMS.batchsize)

train_dataset_theta = [element[0]['theta'] for element in train_dataset]
train_dataset_theta_Y = [element[1] for element in train_dataset_adv]

In [ ]:
# Proportion of data to train and validate
prop_train = 0.90 
prop_validate = 1 - prop_train 

if prop_train == 0:
    print('You need to train on data.')
    prop_train = 0.50
    prop_validate = 0.50
    
if prop_validate == 0:
    print('You need to validate.')
    prop_train = 0.50
    prop_validate = 0.50

### Initialize loss, accuracy and optimizers

In [ ]:
loss_class_fn = tf.keras.losses.CategoricalCrossentropy()
loss_adv_fn = tf.keras.losses.CategoricalCrossentropy()

# class_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
# adv_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

class_acc_metric = tf.keras.metrics.CategoricalAccuracy()
adv_acc_metric = tf.keras.metrics.CategoricalAccuracy()


optimizer_class=tf.keras.optimizers.Adam(lr=HPARAMS.classifier_lr)
optimizer_adv=tf.keras.optimizers.Adam(lr=HPARAMS.adversary_lr)
optimizer=tf.keras.optimizers.Adam(lr=HPARAMS.classifier_lr)

## Build and combine models

### Inputs for classifier model

In [ ]:
classifier_inputs = []
xlayers = []

n = len(layer_names)

for i in range(n):

    if('theta' in layer_names[i]):
        
        inpt = tf.keras.Input(shape = (1,), name=layer_names[i])

        x = tf.keras.layers.Flatten()(inpt)
        classifier_inputs.append(inpt)
        xlayers.append(x)

        x_dense = tf.keras.layers.Dense(64, activation='relu')(x)
        xlayers.append(x_dense)

    else:

        inpt = tf.keras.Input(shape = (10,1), name=layer_names[i])
        x = tf.keras.layers.Flatten()(inpt)

        classifier_inputs.append(inpt)
        xlayers.append(x) 

if(n > 1):
    concatenated_layers_classifier = tf.keras.layers.concatenate(inputs=xlayers, axis=-1)

if(n==1):
    classifier_inputs = classifier_inputs[0]

classifier_inputs_dict = {c.name:c for c in classifier_inputs}

### Inputs for adversary model

In [ ]:
input_from_model = tf.keras.layers.Input(shape = (2,), name='Input')
x_input = tf.keras.layers.Flatten()(input_from_model)

feature_input = tf.keras.layers.Input(shape = (1,), name='theta_2')
x_feature = tf.keras.layers.Flatten()(feature_input)

concatenated_layers_adv = tf.keras.layers.concatenate(inputs=[x_feature,x_input], axis=-1)
adversary_inputs = [input_from_model, x_feature]

## Build Classifier and Adversary

In [ ]:
classifier = build_classifier(classifier_inputs, concatenated_layers_classifier)
adversary = build_adversary(adversary_inputs, concatenated_layers_adv)

classifier_output = classifier(classifier_inputs)
adversary_output = adversary(adversary_inputs)

## Build Combined Model

In [ ]:
model_inputs = {**classifier_inputs_dict,
            'Input':input_from_model,
            'theta_2':feature_input,
    }
model_inputs

In [ ]:
model = tf.keras.Model(
    inputs=model_inputs,
    outputs={'c_out':classifier_output,
             'a_out':adversary_output,
            },
    
    name='Classifier_and_Adversary',
    
)

In [ ]:
model.summary()

## Custom Training

Train adversary for 20 epochs

In [ ]:

classifier.compile(
    loss=loss_class_fn,
    optimizer=optimizer_class,
)

adversary.compile(
    loss=loss_adv_fn,
    optimizer=optimizer_adv,
)

model.compile(optimizer=optimizer)

In [ ]:

classifierEpochs = 1
adversaryEpochs=10

if(classifierOnly):
    classifierEpochs = 10

### Train Classifier

In [ ]:
classifier.fit(X_train, Y_train, epochs=classifierEpochs)

### Train Adversary

In [ ]:

if(not classifierOnly):
    adversary.fit([classifier(X_train), X_train['theta']], theta_labels, epochs=adversaryEpochs, batch_size=HPARAMS.batchsize)

## Training Loop

In [ ]:
# to not get thousands of warnings about gradients
tf.get_logger().setLevel('ERROR')

In [ ]:
trainLambda = False

if(trainLambda):

    trainable_lambda = tf.Variable(2., dtype='float32', name='lambda',
                                   constraint=lambda x: tf.clip_by_value(x, 1, np.infty))

In [ ]:
 
def calc_loss_total(loss_c, loss_a, Lambda):

    # loss_c  - loss of classifier
    # loss_a  - loss of adversary
    
    loss_c = tf.cast(loss_c, 'float32')
    loss_a = tf.cast(loss_a, 'float32')

    L = tf.reduce_mean(tf.math.abs(tf.reduce_mean(loss_c - Lambda * loss_a)))
    
    return L


In [ ]:
@tf.function
def train_step(X_c, Y_c, X_a, Y_a, classifier, adversary, model,
               acc, Lfn_a, Lfn_c, optimizer, Lambda, trainLambda=False):
    
    # X_c : classifier dataset for batch
    # Y_c : classifier labels for batch
    # X_a : adversary dataset for batch
    # Y_a : adversary labels for batch
    # Lfn_c : classifier loss function
    # Lfn_a : adversary loss function
    # acc : accuracy metric
    
    
    with tf.GradientTape(persistent=True) as tape_class:
        
        # adversary output and labels for the batch
        logits_adv = adversary([classifier(X_c), X_a])
    
        # classifier output for the batch
        logits_class = classifier(X_c) 

        # get classifier loss
#         loss_class = Lfn_c(logits_class, Y_c)
        loss_class = Lfn_c(Y_c, logits_class)

        # get adversary loss
        loss_adv = Lfn_a(Y_a, logits_adv)
        
        tape_class.watch(loss_class)
        tape_class.watch(loss_adv)
        
        # get combined loss
        loss_total = calc_loss_total(loss_class, loss_adv, Lambda)
        
        tape_class.watch(loss_total)
        tape_class.watch(model.trainable_weights)

        # get and apply gradients
        grads = tape_class.gradient(loss_total, model.trainable_weights)
        if(trainLambda): grads_l = tape_class.gradient(loss_total, [Lambda])
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if(trainLambda): optimizer.apply_gradients(zip(grads_l, [Lambda]))

        acc.update_state(tf.reshape(Y_c, (*Y_c.shape, 1)), tf.reshape(logits_class, (*logits_class.shape, 1)))

        tape_class.reset()
    
    return loss_total, loss_class, loss_adv, acc.result()

In [ ]:
metrics = {}
metrics['loss_class'] = list([])
metrics['loss_adv'] = list([])
metrics['loss_total'] = list([])
metrics['acc_class'] = list([])
metrics['acc_adv'] = list([])
metrics['lambda'] = list([])
metrics['epoch'] = list([])

In [ ]:
nBatchSteps = nEvents*HPARAMS.valSplit / HPARAMS.batchsize
loss_batch = 0
loss_batch_a = 0
loss_batch_c = 0
acc_batch = 0
total_epochs=1

In [ ]:

if (not classifierOnly):

    last_epoch=time.time()
    start_training = time.time()
 
    nEpochs = 1

    nUpdates_Classifier = HPARAMS.nUpdates_Classifier
#     nUpdates_Classifier = 20

    for ii in range(nEpochs):

        start_update=time.time()

        for epoch in range(nUpdates_Classifier):
            
            loss_batch = 0
            loss_batch_c = 0
            loss_batch_a = 0
            acc_batch = 0

            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                
                loss_total, loss_class, loss_adv, acc_class = train_step(x_batch_train,
                                                                         y_batch_train, 
                                                                         train_dataset_theta[step],
                                                                         train_dataset_theta_Y[step],
                                                                         classifier, adversary, model,
                                                                         class_acc_metric,
                                                                         loss_class_fn, 
                                                                         loss_adv_fn,
                                                              optimizer, HPARAMS.Lambda, trainLambda)
                
                
            
                    
                
                
                loss_batch += loss_total.numpy()
                loss_batch_c += loss_class.numpy()
                loss_batch_a += loss_adv.numpy()
                acc_batch += acc_class.numpy()
                
    


            for step, (x_batch, y_batch) in enumerate(valid_dataset):


                val_logits = classifier(x_batch, training = False)
                
                class_acc_metric.update_state(tf.reshape(y_batch, (*y_batch.shape,1)),
                                              tf.reshape(val_logits, (*val_logits.shape,1)))

            
    
            metrics['acc_class'].append(class_acc_metric.result().numpy())
            class_acc_metric.reset_states()

            loss_epoch   = loss_batch   / nBatchSteps
            loss_epoch_c = loss_batch_c / nBatchSteps
            loss_epoch_a = loss_batch_a / nBatchSteps
            acc_epoch    = acc_batch    / nBatchSteps
            

                
            print('Epoch {0:0.0f}   Loss: {1:0.4f}  Loss class: {2:0.4f}  Loss adv: {3:0.4f}  val acc: {4:0.4f}  {5}'.format(
                total_epochs,
                loss_epoch,
                loss_epoch_c,
                loss_epoch_a,
                acc_epoch,
                printTime(time.time() - last_epoch, returnString=True)

            ))
            
            metrics['loss_total'].append(loss_epoch)
            metrics['loss_class'].append(loss_epoch_c)
            metrics['loss_adv'].append(loss_epoch_a)
            metrics['acc_class'].append(acc_epoch)
            metrics['epoch'].append(total_epochs)
            
            
                
                

            if(trainLambda): print('lambda {0:0.4f}'.format(trainable_lambda.numpy()))

            last_epoch = time.time()

            if(trainLambda): metrics['lambda'].append(trainable_lambda.numpy())

            print('train adversary ', end='')
            adversary.fit([classifier(X_train), X_train['theta']], theta_labels, epochs=1, batch_size=HPARAMS.batchsize)

            
            total_epochs += 1

            




        end_update = time.time()
        printTime(end_update - start_update)

            

        print()

    print()
    end_training=time.time()
    print('\nTotal time: ', end='')
    printTime(end_training - start_training)

    


## Save Models

In [ ]:
# create model names 

model_string = ''
for name in layer_names:
    model_string += '_' + name
if(classifierOnly):
    model_string += '_classifierOnly'
model_name = 'model' + model_string
model_string

In [ ]:
# save models 

tf.keras.models.save_model(model, f'models/{model_string}.h5')
print('saving '+'models/'+model_name+'.h5')
if (not classifierOnly):
    tf.keras.models.save_model(classifier, f'models/{model_name}_classifier.h5')
    tf.keras.models.save_model(adversary, f'models/{model_name}_adversary.h5')

    print(f'saving models/{model_name}_classifier.h5')
    print(f'saving models/{model_name}_adversary.h5')

# Plots

## Loss

In [ ]:

cmap = plt.get_cmap('Accent')
colors = np.linspace(0,1,9)

In [ ]:
metrics['epoch']

In [ ]:
fig = plt.figure(figsize=(7, 5))


lw=3


plt.plot(metrics['epoch'], metrics['loss_total'], lw=lw, color=cmap(colors[0]), label=r'$\mathcal{L}$')
plt.plot(metrics['epoch'], metrics['loss_class'], lw=lw, color=cmap(colors[1]), label=r'$\mathcal{L}_\mathcal{class}$')
plt.plot(metrics['epoch'], HPARAMS.Lambda*np.array(metrics['loss_adv']), lw=lw, color=cmap(colors[2]), label=r'$\mathcal{\lambda} \mathcal{L}_\mathcal{adv}$')

# plt.plot(x1, HPARAMS.Lambda*np.average(loss_avg_a, axis=0), lw=lw, color=cmap(colors[2]), label=r'$\lambda \mathcal{L_{adv}}$')
# plt.plot(x1, np.average(loss_avg_c, axis=0) - 100*np.average(loss_avg_a, axis=0), lw=lw, color=cmap(colors[3]), label='Loss: Total')


plt.legend(fontsize=15)
plt.ylim(0,2)
plt.xticks(metrics['epoch'], fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Steps', fontsize=15)
plt.ylabel('Loss', fontsize=15)
plt.show()



In [ ]:
# fig = plt.figure(figsize=(10,7.5))
# x_epochs = np.arange(len(metrics['loss_total']), dtype=int) + 1

# lw=3


# plt.plot(x_epochs, metrics['loss_total'], lw=lw, color=cmap(colors[0]), label='Loss: Total')
# # plt.plot(x_epochs, metrics['loss_class'], lw=lw, color=cmap(colors[1]), label='Loss: Classifier')
# # plt.plot(x_epochs, HPARAMS.Lambda*np.array(metrics['loss_adv']), lw=lw, color=cmap(colors[2]), label='Loss: Adversary')
# plt.legend(fontsize=15)
# # plt.ylim(-0.1,10.1)
# plt.xticks(x_epochs[::20], fontsize=12)
# plt.yticks(fontsize=12)
# plt.xlabel('Steps', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.show()

# fig = plt.figure(figsize=(7,5))
# x_epochs = np.arange(len(metrics['acc_class']), dtype=int) + 1

# lw=3


# plt.plot(x_epochs, metrics['acc_class'], lw=lw, color=cmap(colors[4]), label='Val Accuracy')
# # plt.plot(x_epochs, metrics['loss_class'], lw=lw, color=cmap(colors[1]), label='Loss: Classifier')
# # plt.plot(x_epochs, HPARAMS.Lambda*np.array(metrics['loss_adv']), lw=lw, color=cmap(colors[2]), label='Loss: Adversary')
# plt.legend(fontsize=15)
# # plt.ylim(-0.1,10.1)
# plt.xticks(x_epochs[::40], fontsize=12)
# plt.yticks(fontsize=12)
# plt.xlabel('Steps', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.show()

### Variable dictionaries for plotting

In [ ]:
xvars = {name:X_train[name].squeeze() for name in layer_names}

for key in xvars.keys():
    
    if(len(xvars[key].shape) > 1):
        xvars[key] = np.average(xvars[key], axis=1)
        

### Get Model Predictions

In [ ]:
predict = classifier(X_train)
predict_adv = adversary([predict, X_train['theta']])

### Uncomment to load saved models instead

In [ ]:
# loaded_model = tf.keras.models.load_model('models/model_theta_tau1_tau2.h5')
# loaded_classifier = tf.keras.models.load_model('models/model_theta_tau1_tau2_classifier.h5')
# loaded_adversary = tf.keras.models.load_model('models/model_theta_tau1_tau2_adversary.h5')

# predict = loaded_classifier(X_train)
# predict_adv = loaded_adversary([predict, X_train['theta']])

## Predictions

In [ ]:
sig = predict[Y_train[:,1]==1][:,1].numpy()
bkg = predict[Y_train[:,0]==1][:,1].numpy()

bins = np.linspace(0, 1, 20)

plt.hist(sig, bins, alpha=0.5, label='Signal', density=True)
plt.hist(bkg, bins, alpha=0.5, label='Background', density=True)
# plt.ylim(0,25000)
# plt.yscale('log')
plt.legend(loc='upper center')
plt.xlabel('Prediction')
plt.ylabel('Density')
plt.show()

## Plot Model Response

In [ ]:
rows = Nlist // 2 + Nlist % 2
cols = 2
fig, ax = plt.subplots(rows,cols, figsize=(10,4*rows))

Title = 'Model Inputs:'
for name in layer_names:
    Title += ', ' + xlabels[name]
    
Title = Title.replace(':,', ':')



xvals_list = []
yvals_list = []
yerrs_list = []

xvals_list_bkg = []
yvals_list_bkg = []
yerrs_list_bkg = []

for name in layer_names:
    
    x, y, e = get_prediction_yvalues(xvars[name], predict[:,1])
    xb, yb, eb = get_prediction_yvalues(xvars[name], predict[:,0])
    
    xvals_list.append(x)
    yvals_list.append(y)
    yerrs_list.append(e)

    xvals_list_bkg.append(xb)
    yvals_list_bkg.append(yb)
    yerrs_list_bkg.append(eb)
    
    
    
if (Nlist > 0):
    if(Nlist < 2):
        ax1 = ax[0]
    elif(Nlist < 3):
        ax1 = ax[0]
    else:
        ax1 = ax[0][0]
        
        
if (Nlist > 1):
    if(Nlist < 3):
        ax2 = ax[1]
    else:
        ax2 = ax[0][1]
        

if (Nlist > 2): ax3 = ax[1][0]
if (Nlist > 3): ax4 = ax[1][1]
if (Nlist > 4): ax5 = ax[2][0]
if (Nlist > 5): ax6 = ax[2][1]
    
    
fmt = ''
lw = 3

if (Nlist > 0):
    
    ax1.errorbar(xvals_list[0], yvals_list[0], yerr=yerrs_list[0], lw=lw, fmt=fmt)
    ax1.errorbar(xvals_list_bkg[0], yvals_list_bkg[0], yerr=yerrs_list_bkg[0], lw=lw, fmt=fmt)
    ax1.set_xlabel(xlabels[layer_names[0]])
    ax1.set_ylabel(r'Output')    
    
if (Nlist > 1):
    
    ax2.errorbar(xvals_list[1], yvals_list[1], yerr=yerrs_list[1], lw=lw, fmt=fmt)
    ax2.errorbar(xvals_list_bkg[1], yvals_list_bkg[1], yerr=yerrs_list_bkg[1], lw=lw, fmt=fmt)
    ax2.set_xlabel(xlabels[layer_names[1]])
    ax2.set_ylabel(r'Output')
    
if (Nlist > 2):
    
    ax3.errorbar(xvals_list[2], yvals_list[2], yerr=yerrs_list[2], lw=lw, fmt=fmt)
    ax3.errorbar(xvals_list_bkg[2], yvals_list_bkg[2], yerr=yerrs_list_bkg[2], lw=lw, fmt=fmt)
    ax3.set_xlabel(xlabels[layer_names[2]])
    ax3.set_ylabel(r'Output')

if (Nlist > 3):
    
    ax4.errorbar(xvals_list[3], yvals_list[3], yerr=yerrs_list[3], lw=lw, fmt=fmt)
    ax4.errorbar(xvals_list_bkg[3], yvals_list_bkg[3], yerr=yerrs_list_bkg[3], lw=lw, fmt=fmt)
    ax4.set_xlabel(xlabels[layer_names[3]])
    ax4.set_ylabel(r'Output')
    
if (Nlist > 4):
    
    ax5.errorbar(xvals_list[4], yvals_list[4], yerr=yerrs_list[4], lw=lw, fmt=fmt)
    ax5.errorbar(xvals_list_bkg[4], yvals_list_bkg[4], yerr=yerrs_list_bkg[4], lw=lw, fmt=fmt)
    ax5.set_xlabel(xlabels[layer_names[4]])
    ax5.set_ylabel(r'Output')
    
if (Nlist > 5):
    
    ax6.errorbar(xvals_list[5], yvals_list[5], yerr=yerrs_list[5], lw=lw, fmt=fmt)
    ax6.errorbar(xvals_list_bkg[5], yvals_list_bkg[5], yerr=yerrs_list_bkg[5], lw=lw, fmt=fmt)
    ax6.set_xlabel(xlabels[layer_names[5]])
    ax6.set_ylabel(r'Output')   

fig.legend(['Signal', 'Background'])




figname = 'plots/Lambda'+str(HPARAMS.Lambda)+'_scatterplots.png'
plt.savefig(figname)
print('saving '+figname)

plt.suptitle(Title, fontsize=25)
# plt.tight_layout()


plt.show()

In [ ]:
fig, ax = plt.subplots(rows,cols, figsize=(10,4*rows))

plt.suptitle(Title, fontsize=25)

xvals_list = []
yvals_list = []
yerrs_list = []

xvals_list_bkg = []
yvals_list_bkg = []
yerrs_list_bkg = []


yrange = [-0.05,1.05]

for name in layer_names:
    
    x, y, e = get_prediction_yvalues(xvars[name], predict[:,1], norm=True)
    xb, yb, eb = get_prediction_yvalues(xvars[name], predict[:,0], norm=True)
    
    xvals_list.append(x)
    yvals_list.append(y)
    yerrs_list.append(e)

    xvals_list_bkg.append(xb)
    yvals_list_bkg.append(yb)
    yerrs_list_bkg.append(eb)
    
    
    
if (Nlist > 0):
    if(Nlist < 2):
        ax1 = ax[0]
    elif(Nlist < 3):
        ax1 = ax[0]
    else:
        ax1 = ax[0][0]
        
        
if (Nlist > 1):
    if(Nlist < 3):
        ax2 = ax[1]
    else:
        ax2 = ax[0][1]
        

if (Nlist > 2): ax3 = ax[1][0]
if (Nlist > 3): ax4 = ax[1][1]
if (Nlist > 4): ax5 = ax[2][0]
if (Nlist > 5): ax6 = ax[2][1]

if (Nlist > 0):
    
    ax1.errorbar(xvals_list[0], yvals_list[0], yerr=yerrs_list[0], lw=lw, fmt=fmt)
    ax1.errorbar(xvals_list_bkg[0], yvals_list_bkg[0], yerr=yerrs_list_bkg[0], lw=lw, fmt=fmt)
    ax1.set_xlabel(xlabels[layer_names[0]])
    ax1.set_ylabel(r'Normalized Output')
    ax1.set_ylim(yrange)
    
if (Nlist > 1):
    
    ax2.errorbar(xvals_list[1], yvals_list[1], yerr=yerrs_list[1], lw=lw, fmt=fmt)
    ax2.errorbar(xvals_list_bkg[1], yvals_list_bkg[1], yerr=yerrs_list_bkg[1], lw=lw, fmt=fmt)
    ax2.set_xlabel(xlabels[layer_names[1]])
    ax2.set_ylabel(r'Normalized Output')
    ax2.set_ylim(yrange)
    
if (Nlist > 2):
    
    ax3.errorbar(xvals_list[2], yvals_list[2], yerr=yerrs_list[2], lw=lw, fmt=fmt)
    ax3.errorbar(xvals_list_bkg[2], yvals_list_bkg[2], yerr=yerrs_list_bkg[2], lw=lw, fmt=fmt)
    ax3.set_xlabel(xlabels[layer_names[2]])
    ax3.set_ylabel(r'Normalized Output')
    ax3.set_ylim(yrange)

if (Nlist > 3):
    
    ax4.errorbar(xvals_list[3], yvals_list[3], yerr=yerrs_list[3], lw=lw, fmt=fmt)
    ax4.errorbar(xvals_list_bkg[3], yvals_list_bkg[3], yerr=yerrs_list_bkg[3], lw=lw, fmt=fmt)
    ax4.set_xlabel(xlabels[layer_names[3]])
    ax4.set_ylabel(r'Normalized Output')
    ax4.set_ylim(yrange)
    
if (Nlist > 4):
    
    ax5.errorbar(xvals_list[4], yvals_list[4], yerr=yerrs_list[4], lw=lw, fmt=fmt)
    ax5.errorbar(xvals_list_bkg[4], yvals_list_bkg[4], yerr=yerrs_list_bkg[4], lw=lw, fmt=fmt)
    ax5.set_xlabel(xlabels[layer_names[4]])
    ax5.set_ylabel(r'Normalized Output')
    ax5.set_ylim(yrange)
    
if (Nlist > 5):
    
    ax6.errorbar(xvals_list[5], yvals_list[5], yerr=yerrs_list[5], lw=lw, fmt=fmt)
    ax6.errorbar(xvals_list_bkg[5], yvals_list_bkg[5], yerr=yerrs_list_bkg[5], lw=lw, fmt=fmt)
    ax6.set_xlabel(xlabels[layer_names[5]])
    ax6.set_ylabel(r'Normalized Output')
    ax6.set_ylim(yrange)

fig.legend(['Signal', 'Background'])
# plt.tight_layout()



figname = 'plots/Lambda'+str(HPARAMS.Lambda)+'_scatterplots_norm.png'
plt.savefig(figname)
print('saving '+figname)

plt.show()

## Plot Adversary Output

In [ ]:
rows=2
cols=3

fig, ax = plt.subplots(rows,cols, figsize=(15,7))

if(HPARAMS.thetaBins > 0): plotBin(1, xvars['theta'], predict_adv, rows, cols, ax)
if(HPARAMS.thetaBins > 1): plotBin(2, xvars['theta'], predict_adv, rows, cols, ax)
if(HPARAMS.thetaBins > 2): plotBin(3, xvars['theta'], predict_adv, rows, cols, ax)
if(HPARAMS.thetaBins > 3): plotBin(4, xvars['theta'], predict_adv, rows, cols, ax)
if(HPARAMS.thetaBins > 4): plotBin(5, xvars['theta'], predict_adv, rows, cols, ax)
if(HPARAMS.thetaBins > 5): plotBin(6, xvars['theta'], predict_adv, rows, cols, ax)
    
plt.tight_layout()
figname = 'plots/Adversary_Output.png'
plt.savefig(figname)
print('saving '+figname)
plt.show()

In [ ]:
printTime(time.time()-start_notebook)